<h1 align=center><font size = 5>Assignment: Segmenting and Clustering Neighborhoods in Toronto</font></h1>
<h3 align=center>David Gerardo Luna Gallegos, B.S.ME.E.</h3>

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np 
import folium
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import requests
%matplotlib inline

The dataset was locally downloaded as a csv file using a [wikitable-to-csv converter](https://wikitable2csv.ggor.de/), so it would be much easier to convert into dataframe format.

In [2]:
postal_code_df = pd.read_csv('Toronto_PostalCodes.csv')
postal_code_df = postal_code_df.sort_values('Postal Code').reset_index(drop = True)
print('Initial shape: {}'.format(postal_code_df.shape))
postal_code_df.head()

Initial shape: (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


There are 180 initial features of our 3 attributes. Nevertheless, there are invalid values in the Neighbourhood colu, hence, droping rows including "_Not assinged_"  was required.

In [3]:
postal_code_filtered = postal_code_df[postal_code_df['Borough'] != 'Not assigned']
postal_code_filtered = postal_code_filtered.reset_index(drop = True)
print('Shape after dropping rows: {}'.format(postal_code_filtered.shape))
postal_code_filtered

Shape after dropping rows: (103, 3)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Now that 77 rows were dropped, we encounter that some neighbourhoods are grouped in a single string. That could cause problems with neighborhood location, hence, splitting was required.

In [4]:
for i, nbh in zip(range(len(postal_code_filtered['Neighbourhood'])),postal_code_filtered['Neighbourhood']):

    postal_code_filtered['Neighbourhood'][i] = nbh.split(', ')
    
postal_code_filtered.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"[Malvern, Rouge]"
1,M1C,Scarborough,"[Rouge Hill, Port Union, Highland Creek]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


Neighborhood features are now splitted, but Neighborhood column has to be rearanged along its companion attributes -Postal Code and Borough- in a new dataframe.

In [5]:
index_list = []
nbh_list = []
b_list = []
PC_list = []
    
for nbh, bo, pc in zip(postal_code_filtered['Neighbourhood'],postal_code_filtered['Borough'],postal_code_filtered['Postal Code']):

    if len(nbh) > 1:
        for nbh_value in nbh:
            
            nbh_list.append(nbh_value)
            b_list.append(bo)
            PC_list.append(pc)
            
    else:    
    
        nbh_list.append(nbh[0])
        b_list.append(bo)
        PC_list.append(pc)
    
toronto_nbh = pd.DataFrame(columns = ['Postal Code', 'Borough','Neighbourhood'])
toronto_nbh['Postal Code'] = PC_list
toronto_nbh['Borough'] = b_list
toronto_nbh['Neighbourhood'] = nbh_list

for i, nbh in zip(range(len(nbh_list)), nbh_list):
    
    if nbh[0] == ' ':
        
        toronto_nbh['Neighbourhood'][i] = nbh.replace(' ', '', 1)



print('Shape after rearenging: {}'.format(toronto_nbh.shape))
toronto_nbh.head()

Shape after rearenging: (217, 3)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,Malvern
1,M1B,Scarborough,Rouge
2,M1C,Scarborough,Rouge Hill
3,M1C,Scarborough,Port Union
4,M1C,Scarborough,Highland Creek


In [6]:
print('Unique neighbourhood count: {}'.format(toronto_nbh['Neighbourhood'].nunique()))

Unique neighbourhood count: 208


Duplicated neighbourhood values detected, dropping duplicates is required.

In [7]:
toronto_nbh = toronto_nbh.drop_duplicates('Neighbourhood', keep = 'first').reset_index(drop = True)
print('Shape after dropping duplicates: {}'.format(toronto_nbh.shape))

Shape after dropping duplicates: (208, 3)


With data in the appropiate format, location can now be obtained.

In [8]:
lat_list = []
lng_list = []

geolocator = Nominatim(user_agent = 'toronto_explorer')

for nbh in toronto_nbh['Neighbourhood']:
    try:
        nbh_location = geolocator.geocode('{}, Toronto, Ontario'.format(nbh))
        lat_list.append(nbh_location.latitude)
        lng_list.append(nbh_location.longitude)

    except:

        lat_list.append('Not found')
        lng_list.append('Not found')

toronto_nbh['Latitude'] = lat_list
toronto_nbh['Longitude'] = lng_list  
toronto_nbh.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern,43.8092,-79.2217
1,M1B,Scarborough,Rouge,43.8049,-79.1658
2,M1C,Scarborough,Rouge Hill,43.7803,-79.1305
3,M1C,Scarborough,Port Union,43.7755,-79.135
4,M1C,Scarborough,Highland Creek,43.7901,-79.1733
5,M1E,Scarborough,Guildwood,43.7552,-79.1982
6,M1E,Scarborough,Morningside,43.7826,-79.205
7,M1E,Scarborough,West Hill,43.7689,-79.1873
8,M1G,Scarborough,Woburn,Not found,Not found
9,M1H,Scarborough,Cedarbrae,Not found,Not found


"_Not found_" values are observed, and again, dropping rows is required.

In [9]:
toronto_nbh = toronto_nbh[toronto_nbh['Latitude'] != 'Not found']
print('Shape after dropping rows: {}'.format(toronto_nbh.shape))
toronto_nbh = toronto_nbh.reset_index(drop = True)
toronto_nbh.head(10)

Shape after dropping rows: (192, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern,43.8092,-79.2217
1,M1B,Scarborough,Rouge,43.8049,-79.1658
2,M1C,Scarborough,Rouge Hill,43.7803,-79.1305
3,M1C,Scarborough,Port Union,43.7755,-79.135
4,M1C,Scarborough,Highland Creek,43.7901,-79.1733
5,M1E,Scarborough,Guildwood,43.7552,-79.1982
6,M1E,Scarborough,Morningside,43.7826,-79.205
7,M1E,Scarborough,West Hill,43.7689,-79.1873
8,M1J,Scarborough,Scarborough Village,43.7437,-79.2116
9,M1K,Scarborough,Kennedy Park,43.7249,-79.254


Neighbourhood visualization by borough.

In [10]:
address = 'Toronto, Ontario'

location = geolocator.geocode(address)
latitude = location.latitude + .1
longitude = location.longitude

borough_colors = cm.tab10(np.linspace(0,1, len(toronto_nbh['Borough'].unique())))
colors_hex = [colors.rgb2hex(color) for color in borough_colors]

toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 10.5)


for lat, lng, borough, nbhd in zip(toronto_nbh['Latitude'], toronto_nbh['Longitude'], toronto_nbh['Borough'], toronto_nbh['Neighbourhood']):

    i = 0

    while not(borough == toronto_nbh['Borough'].unique()[i]):

        i = i + 1

    label = '{},{}'.format(nbhd, borough)
    label = folium.Popup(label, parse_html = True)

    folium.CircleMarker(location = [lat, lng],
                        radius = 5,
                        popup = label,
                        color = colors_hex[i],
                        fill = True,
                        fill_color = colors_hex[i],
                        fill_opacity = 0.7).add_to(toronto_map)

toronto_map

As we can observe, either dataset or geolocator have erroneous data because segmentation is not heterogeneous. Since only the neighbourhood is required to retrieve data from Foursquare API, this issue is ignored.

In [11]:
CLIENT_ID = 'CNSMKDJHAVHCRJ3NOZLVVB2JKVHKGN11TDJOVN3HUAZG1ARV'
CLIENT_SECRET = 'B5RZBVRMDRJRZE5YPQDMUA3SSY3X2O3LNCIFXPMYME0WZ4MT'
LIMIT = 100
RADIUS = 500
VERSION = '20180605'

venue_neighbourhood_list = []
venue_name_list = []
venue_category_list = []

venues = pd.DataFrame(columns = ['Neighbourhood', 'Venue', 'Category'])

for lat, lng, nbh in zip(toronto_nbh['Latitude'], toronto_nbh['Longitude'], toronto_nbh['Neighbourhood']):

    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                        CLIENT_SECRET,
                                                                                                        VERSION,
                                                                                                        lat,
                                                                                                        lng,
                                                                                                        RADIUS,
                                                                                                        LIMIT)
    request = requests.get(url).json()

    for i in range(len(request['response']['groups'][0]['items'])):
        
        try:
            venue_neighbourhood_list.append(nbh)
            venue_name_list.append(request['response']['groups'][0]['items'][i]['venue']['name'])
            venue_category_list.append(request['response']['groups'][0]['items'][i]['venue']['categories'][0]['name'])
        
        except:
            print('{} not found'.format(nbh))

venues['Neighbourhood'] = venue_neighbourhood_list
venues['Venue'] = venue_name_list
venues['Category'] = venue_category_list
venues.head()

,Neighbourhood,Venue,Category
0,Malvern,Shoppers Drug Mart,Pharmacy
1,Malvern,Subway,Sandwich Place
2,Malvern,A&W,Fast Food Restaurant
3,Malvern,Pizza Pizza,Pizza Place
4,Malvern,Pizza Hut,Pizza Place


With venues category defined in the dataset, one hot encoding can be performed to find siliratities among neighbourhoods.

# One Hot Encoding

In [13]:
categories_one_hot = pd.get_dummies(venues[['Category']], prefix="", prefix_sep="")
categories_one_hot['Neighbourhood'] = venues['Neighbourhood']

rearenged_columns = [categories_one_hot.columns[-1]] + list(categories_one_hot.columns[:-1])

categories_one_hot = categories_one_hot[rearenged_columns]

categories_one_hot = categories_one_hot.groupby('Neighbourhood').mean()

categories_one_hot = categories_one_hot.reset_index()

print(categories_one_hot.shape)
categories_one_hot.head()

(191, 339)


,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorsports Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon /

# _K_-means

_k_-means algorithm is used to cluster similar neighbourhoods.

In [14]:
kclusters = 5

km = KMeans(n_clusters = kclusters, random_state = 0).fit(categories_one_hot.drop('Neighbourhood', axis = 1))
km.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 3, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2,
       0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0,
       0, 0, 0, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 4, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 3, 2, 0,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 3, 2], dtype=int32)

In [15]:
labeled_nbh = pd.DataFrame(columns = ['Neighbourhood', 'Category Label'])
labeled_nbh['Neighbourhood'] = categories_one_hot['Neighbourhood']
labeled_nbh['Category Label'] = km.labels_

nbh_sorted = toronto_nbh.sort_values('Neighbourhood').reset_index(drop = True)

labeled_nbh['Latitude'] = nbh_sorted['Latitude']
labeled_nbh['Longitude'] = nbh_sorted['Longitude']
labeled_nbh['Borough'] = nbh_sorted['Borough']


labeled_nbh.head()

,Neighbourhood,Category Label,Latitude,Longitude,Borough
0,Adelaide,2,43.6508,-79.3779,Downtown Toronto
1,Agincourt,2,43.7854,-79.2785,Scarborough
2,Agincourt North,2,43.808,-79.2664,Scarborough
3,Albion Gardens,2,43.7417,-79.5845,Etobicoke
4,Alderwood,2,43.6017,-79.5452,Etobicoke


In [16]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start =10.45)

color_label = cm.tab10(np.linspace(0,1, kclusters))
color_label = [colors.rgb2hex(color) for color in color_label]

for nbh, lat, lng, lbl, brh in zip(labeled_nbh['Neighbourhood'], labeled_nbh['Latitude'], labeled_nbh['Longitude'], labeled_nbh['Category Label'], labeled_nbh['Borough']):
    
    label = folium.Popup('{},{}, Cluster {}'.format(nbh, brh, lbl), parse_html = True)
    
    folium.CircleMarker(location = [lat, lng],
                        radius = 5,
                        popup = label,
                        color = color_label[lbl],
                        fill = True,
                        fill_color = color_label[lbl],
                        fill_opacity = 0.7).add_to(map_clusters)
map_clusters

In [17]:
cluster_data =  pd.get_dummies(venues[['Category']], prefix="", prefix_sep="")
cluster_data['Category Label'] = labeled_nbh['Category Label']

columns = [cluster_data.columns[-1]] + list(cluster_data.columns[:-1])

cluster_data = cluster_data[columns]


cluster_data.head()

,Category Label,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorsports Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon 

In [18]:
cluster_common_venues = cluster_data.groupby('Category Label').mean().reset_index()

In [19]:

for i in range(kclusters):

    cluster_common_venues.T.drop('Category Label', axis = 0)[[i]].sort_values(i,ascending = False)
    
    print('---Cluster{}---'.format(i))
    
    for category in cluster_common_venues.T.drop('Category Label', axis = 0)[[i]].sort_values(i,ascending = False).index[0:5]:
        
        print(category)
     
    print('')

---Cluster0---
Coffee Shop
Breakfast Spot
Supermarket
Sandwich Place
Pharmacy

---Cluster1---
Park
Discount Store
Warehouse Store
Leather Goods Store
Gym

---Cluster2---
Fast Food Restaurant
Park
Pizza Place
Grocery Store
Restaurant

---Cluster3---
Diner
Ice Cream Shop
Fast Food Restaurant
Pharmacy
Park

---Cluster4---
Automotive Shop
Hardware Store
Coffee Shop
Accessories Store
Nail Salon



As observed in the map above, the majority of neighbourhoods belong to cluster 2, wich are surounded by food venues and parks. That can tell us more about lifestyle and what people usually do or like. But that is beyond the scope of this assignment.